# Hugging Face LLM Course — Learning Curves Reference

Use this notebook as a hands-on companion to the training sections of the 🤗 LLM course. It focuses on:
- **Visualizing Loss & Accuracy**: Understanding how metrics change over time.
- **Diagnosing Issues**: Identifying overfitting, underfitting, and convergence.

In [1]:
!pip install evaluate
!pip install wandb
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


## Understanding Learning Curves

### What are Learning Curves?
Learning curves are visual representations of your model’s performance metrics over time during training. The two most important curves to monitor are:

- **Loss curves**: Show how the model’s error (loss) changes over training steps or epochs
- **Accuracy curves**: Show the percentage of correct predictions over training steps or epochs

### Loss Curves
The loss curve shows how the model’s error decreases over time. In a typical successful training run

- **High initial loss:** The model starts without optimization, so predictions are initially poor
- **Decreasing loss:** As training progresses, the loss should generally decrease
- **Convergence:** Eventually, the loss stabilizes at a low value, indicating that the model has learned the patterns in the data



In [ ]:
import numpy as np
import evaluate
import wandb
import torch
from datasets import load_dataset
from transformers import (
    Trainer,
    AutoTokenizer,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
)

# --- Step 1: Experiment Tracking Setup ---
# Log in to your W&B account (Required only once)
wandb.login()

# Initialize the project and specific run name
wandb.init(project="transformer-fine-tuning", name="bert-mrpc-analysis")

# --- Step 2: Load Dataset and Model Checkpoint ---
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# --- Step 3: Data Preprocessing ---
def tokenize_function(example):
    """
    Tokenizes pairs of sentences for the MRPC task.
    Truncation is enabled to ensure sentences fit the model's max length.
    """
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Map the tokenization function over the entire dataset in batches
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Data collator handles dynamic padding for batches during training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Step 4: Define Evaluation Metric ---
def compute_metrics(eval_preds):
    """
    Computes accuracy and f1 score for the MRPC dataset.
    Logic: Converts logits to class predictions using argmax.
    """
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# --- Step 5: Initialize the Model ---
# Loading a pre-trained BERT model with a classification head
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# --- Step 6: Define Training Configuration ---
training_args = TrainingArguments(
    output_dir="./result",

    # Evaluation strategy
    eval_strategy="steps",       # Evaluate the model every X steps
    eval_steps=50,               # Perform evaluation every 50 steps

    # Saving and Logging
    save_steps=100,              # Save a model checkpoint every 100 steps
    logging_steps=10,            # Log training metrics every 10 steps

    # Hyperparameters
    num_train_epochs=3,          # Number of full passes through the training data
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    # Integration
    report_to="wandb",           # Automatically send logs to Weights & Biases
)

# --- Step 7: Initialize the Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

# --- Step 8: Start Training ---
print("Training is starting... Monitoring via Weights & Biases.")
trainer.train()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khaledtelbahnasy (khaledtelbahnasy-mansoura-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training is starting... Monitoring via Weights & Biases.


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.588800,0.549081,0.723039,0.777120
100,0.496200,0.436637,0.796569,0.864600
150,0.552400,0.542478,0.725490,0.832836
200,0.420700,0.416661,0.828431,0.877622
250,0.328400,0.437438,0.848039,0.893103
300,0.291500,0.424627,0.840686,0.892206
350,0.262300,0.386823,0.850490,0.896082
400,0.199600,0.391854,0.862745,0.905724
450,0.337600,0.388351,0.857843,0.900685
500,0.192300,0.534108,0.857843,0.902685


TrainOutput(global_step=690, training_loss=0.33296302980270936, metrics={'train_runtime': 478.8734, 'train_samples_per_second': 22.979, 'train_steps_per_second': 1.441, 'total_flos': 428577075854640.0, 'train_loss': 0.33296302980270936, 'epoch': 3.0})

### Accuracy Curves
The accuracy curve shows the percentage of correct predictions over time. Unlike loss curves, accuracy curves should generally increase as the model learns and can typically include more steps than the loss curve.

- **Start low:** Initial accuracy should be low, as the model has not yet learned the patterns in the data
- **Increase with training:** Accuracy should generally improve as the model learns if it is able to learn the patterns in the data
- **May show plateaus:** Accuracy often increases in discrete jumps rather than smoothly, as the model makes predictions that are close to the true labels


### Convergence
Convergence occurs when the model’s performance stabilizes and the loss and accuracy curves level off. This is a sign that the model has learned the patterns in the data and is ready to be used. In simple terms, we are aiming for the model to converge to a stable performance every time we train it.



## Interpreting Learning Curve Patterns

### Healthy Learning Curves
Characteristics of healthy curves:

- Smooth decline in loss: Both training and validation loss decrease steadily
- Close training/validation performance: Small gap between training and validation metrics
- Convergence: Curves level off, indicating the model has learned the patterns

### Overfitting
Overfitting occurs when the model learns too much from the training data and is unable to generalize to different data (represented by the validation set).

#### Symptoms:

- Training loss continues to decrease while validation loss increases or plateaus
- Large gap between training and validation accuracy
- Training accuracy much higher than validation accuracy
#### Solutions for overfitting:

- Regularization: Add dropout, weight decay, or other regularization techniques
- Early stopping: Stop training when validation performance stops improving
- Data augmentation: Increase training data diversity
- Reduce model complexity: Use a smaller model or fewer parameters



In [2]:
import numpy as np
import evaluate
import wandb
import torch
from datasets import load_dataset
from transformers import (
    Trainer,
    AutoTokenizer,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    EarlyStoppingCallback
)

# --- Step 1: Experiment Tracking Setup ---
# Log in to your W&B account (Required only once)
wandb.login()

# Initialize the project and specific run name
wandb.init(project="transformer-fine-tuning", name="bert-mrpc-analysis")

# --- Step 2: Load Dataset and Model Checkpoint ---
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# --- Step 3: Data Preprocessing ---
def tokenize_function(example):
    """
    Tokenizes pairs of sentences for the MRPC task.
    Truncation is enabled to ensure sentences fit the model's max length.
    """
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Map the tokenization function over the entire dataset in batches
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Data collator handles dynamic padding for batches during training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Step 4: Define Evaluation Metric ---
def compute_metrics(eval_preds):
    """
    Computes accuracy and f1 score for the MRPC dataset.
    Logic: Converts logits to class predictions using argmax.
    """
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# --- Step 5: Initialize the Model ---
# Loading a pre-trained BERT model with a classification head
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# --- Step 6: Define Training Configuration ---
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=10,
)

# --- Step 7: Initialize the Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# --- Step 8: Start Training ---
print("Training is starting... Monitoring via Weights & Biases.")
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khaledtelbahnasy (khaledtelbahnasy-mansoura-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training is starting... Monitoring via Weights & Biases.


Step,Training Loss,Validation Loss,Accuracy,F1
100,No log,0.558097,0.708333,0.821589
200,No log,0.508256,0.779412,0.850000
300,No log,0.533536,0.784314,0.853333
400,No log,0.525624,0.781863,0.847863
500,0.546800,0.552550,0.823529,0.871429


TrainOutput(global_step=500, training_loss=0.5467985229492187, metrics={'train_runtime': 312.1611, 'train_samples_per_second': 117.503, 'train_steps_per_second': 14.704, 'total_flos': 147161302151040.0, 'train_loss': 0.5467985229492187, 'epoch': 1.0893246187363834})

### Underfitting
Underfitting occurs when the model is too simple to capture the underlying patterns in the data. This can happen for several reasons:
- The model is too small or lacks capacity to learn the patterns
- The learning rate is too low, causing slow learning
- The dataset is too small or not representative of the problem
- The model is not properly regularized

#### Symptoms:

- Both training and validation loss remain high
- Model performance plateaus early in training
- Training accuracy is lower than expected
#### Solutions for underfitting:

- Increase model capacity: Use a larger model or more parameters
- Train longer: Increase the number of epochs
- Adjust learning rate: Try different learning rates
- Check data quality: Ensure your data is properly preprocessed


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    -num_train_epochs=5,
    +num_train_epochs=10,
)

### Erratic Learning Curves
Erratic learning curves occur when the model is not learning effectively. This can happen for several reasons:

- The learning rate is too high, causing the model to overshoot the optimal parameters
- The batch size is too small, causing the model to learn slowly
- The model is not properly regularized, causing it to overfit to the training data
- The dataset is not properly preprocessed, causing the model to learn from noise

#### Symptoms:

- Frequent fluctuations in loss or accuracy
- Curves show high variance or instability
- Performance oscillates without clear trend

#### Solutions for erratic curves:

- Lower learning rate: Reduce step size for more stable training
- Increase batch size: Larger batches provide more stable gradients
- Gradient clipping: Prevent exploding gradients
- Better data preprocessing: Ensure consistent data quality


### 💡 Key Takeaways:

- Learning curves are essential tools for understanding model training progress
- Monitor both loss and accuracy curves, but remember they have different characteristics
- Overfitting shows as diverging training/validation performance
- Underfitting shows as poor performance on both training and validation data
- Tools like Weights & Biases make it easy to track and analyze learning curves
- Early stopping and proper regularization can address most common training issues